# Pretrain Checkpoint Landscape Probe Experiment

**Question:** Do landscape probes measured during pretraining (under MLM loss) predict which fine-tuning hyperparameters work best?

**Design:** Load MultiBERTs checkpoints at different pretraining stages, measure landscape probes under MLM loss, then fine-tune each on SST-2 with a hyperparameter grid. Correlate pretrain probes with fine-tuning outcomes.

**Quick test:** 1 checkpoint (step_2000k) x 2 seeds x 16 HP configs = 32 fine-tuning runs (~1 hour on T4)

## Setup

In [ ]:
# Install dependencies (uncomment if running in Colab)
# !pip install torch transformers datasets scipy pandas matplotlib seaborn tqdm scikit-learn

In [ ]:
# Clone repo (uncomment if running in Colab)
# !git clone https://github.com/<your-username>/hedgehog.git
# %cd hedgehog/landscape-probes

In [ ]:
import os
import sys
import json
import time
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from scipy import stats

# If running from the landscape-probes directory
if os.path.basename(os.getcwd()) != 'landscape-probes':
    # Try to find it
    for candidate in ['landscape-probes', 'hedgehog/landscape-probes']:
        if os.path.isdir(candidate):
            os.chdir(candidate)
            break

sys.path.insert(0, '.')

print(f"Working directory: {os.getcwd()}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB")

## Import experiment modules

In [ ]:
from probes.landscape_probes import LandscapeProbes
from experiments.pretrain_probe_sweep import (
    MLMCriterion,
    load_mlm_data,
    load_finetune_data,
    measure_pretrain_probes,
    finetune_checkpoint,
)

print("All modules imported successfully.")

## Configuration

Quick test: 1 checkpoint x 2 seeds = 2 pretrained models, each fine-tuned with 16 HP configs.

In [ ]:
# --- Quick test config ---
CHECKPOINT_STEPS = ["step_2000k"]  # Just the final checkpoint
SEEDS = [0, 1]                      # 2 seeds

# --- Full experiment (uncomment when ready) ---
# CHECKPOINT_STEPS = ["step_200k", "step_600k", "step_1000k", "step_2000k"]
# SEEDS = [0, 1, 2, 3, 4]

# HP grid (same as original experiments)
LEARNING_RATES = [1e-5, 2e-5, 5e-5, 1e-4]
WARMUP_RATIOS = [0.0, 0.1]
WEIGHT_DECAYS = [0.0, 0.01]

TASK = "sst2"
BATCH_SIZE = 16
NUM_EPOCHS = 1
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
OUTPUT_DIR = "./results_pretrain"
N_HUTCHINSON = 50

n_hp = len(LEARNING_RATES) * len(WARMUP_RATIOS) * len(WEIGHT_DECAYS)
n_checkpoints = len(SEEDS) * len(CHECKPOINT_STEPS)
total_runs = n_checkpoints * n_hp

print(f"Checkpoints: {CHECKPOINT_STEPS}")
print(f"Seeds: {SEEDS}")
print(f"HP configs: {n_hp}")
print(f"Total fine-tuning runs: {total_runs}")
print(f"Estimated time: ~{total_runs * 3 / 60:.1f} hours on T4")

## Load data

Load MLM data (WikiText-2) for pretrain probe measurement, and SST-2 for fine-tuning. This only needs to happen once.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

print("Loading MLM data (WikiText-2)...")
mlm_loader = load_mlm_data(tokenizer, batch_size=BATCH_SIZE)

print(f"Loading {TASK} data...")
train_loader, val_loader, num_labels = load_finetune_data(
    tokenizer, TASK, batch_size=BATCH_SIZE,
)

print("Data loaded.")

## Run experiment

For each checkpoint:
1. Measure 6 landscape probes under MLM loss
2. Fine-tune on SST-2 with each HP config
3. Save results incrementally

In [ ]:
os.makedirs(OUTPUT_DIR, exist_ok=True)
output_path = os.path.join(OUTPUT_DIR, "pretrain_probe_results.json")

# Resume support: load existing results
results = []
completed = set()

if os.path.exists(output_path):
    with open(output_path) as f:
        results = json.load(f)
    for r in results:
        completed.add((r['seed'], r['checkpoint_step'], r['lr'],
                       r['warmup_ratio'], r['weight_decay']))
    print(f"Resuming: {len(completed)} runs already completed, {total_runs - len(completed)} remaining")
else:
    print(f"Starting fresh: {total_runs} runs to go")

pretrain_probes_cache = {}
run_count = 0
experiment_start = time.time()

for seed in SEEDS:
    for step in CHECKPOINT_STEPS:
        checkpoint_name = f"google/multiberts-seed_{seed}-{step}"
        cache_key = (seed, step)

        # Skip if all HP configs done for this checkpoint
        all_done = all(
            (seed, step, lr, warmup, wd) in completed
            for lr in LEARNING_RATES
            for warmup in WARMUP_RATIOS
            for wd in WEIGHT_DECAYS
        )
        if all_done:
            run_count += n_hp
            print(f"Skipping {checkpoint_name} (all configs done)")
            continue

        print(f"\n{'=' * 60}")
        print(f"Checkpoint: {checkpoint_name}")
        print(f"{'=' * 60}")

        # --- Measure pretrain probes (once per checkpoint) ---
        if cache_key not in pretrain_probes_cache:
            try:
                probes = measure_pretrain_probes(
                    checkpoint_name, mlm_loader, DEVICE,
                    n_hutchinson_samples=N_HUTCHINSON,
                )
                pretrain_probes_cache[cache_key] = probes
                print(f"  Probes: grad_norm={probes['gradient_norm']:.4f}, "
                      f"trace={probes['hutchinson_trace']:.4f}, "
                      f"top_eig={probes['top_eigenvalue']:.4f}")
            except Exception as e:
                print(f"  ERROR measuring probes: {e}")
                pretrain_probes_cache[cache_key] = {
                    k: float('nan') for k in [
                        'gradient_norm', 'gradient_variance', 'sam_sharpness',
                        'hutchinson_trace', 'top_eigenvalue', 'loss_at_init',
                    ]
                }

        # --- Fine-tune with each HP config ---
        for lr in LEARNING_RATES:
            for warmup in WARMUP_RATIOS:
                for wd in WEIGHT_DECAYS:
                    run_count += 1
                    config_key = (seed, step, lr, warmup, wd)

                    if config_key in completed:
                        continue

                    print(f"  [{run_count}/{total_runs}] lr={lr}, warmup={warmup}, wd={wd}", end=" ")

                    try:
                        ft_result = finetune_checkpoint(
                            checkpoint_name, train_loader, val_loader,
                            num_labels, lr=lr, warmup_ratio=warmup,
                            weight_decay=wd, num_epochs=NUM_EPOCHS,
                            device=DEVICE,
                        )
                        print(f"-> acc={ft_result['val_accuracy']:.4f}")
                    except Exception as e:
                        print(f"-> ERROR: {e}")
                        ft_result = {
                            'val_accuracy': float('nan'),
                            'final_train_loss': float('nan'),
                        }

                    pretrain_probes = pretrain_probes_cache[cache_key]
                    result = {
                        'seed': seed,
                        'checkpoint_step': step,
                        'checkpoint_name': checkpoint_name,
                        'task': TASK,
                        'lr': lr,
                        'warmup_ratio': warmup,
                        'weight_decay': wd,
                        'batch_size': BATCH_SIZE,
                        'num_epochs': NUM_EPOCHS,
                        'val_accuracy': ft_result['val_accuracy'],
                        'final_train_loss': ft_result['final_train_loss'],
                        'pretrain_gradient_norm': pretrain_probes['gradient_norm'],
                        'pretrain_gradient_variance': pretrain_probes['gradient_variance'],
                        'pretrain_sam_sharpness': pretrain_probes['sam_sharpness'],
                        'pretrain_hutchinson_trace': pretrain_probes['hutchinson_trace'],
                        'pretrain_top_eigenvalue': pretrain_probes['top_eigenvalue'],
                        'pretrain_loss': pretrain_probes['loss_at_init'],
                    }

                    results.append(result)

                    # Save incrementally
                    with open(output_path, 'w') as f:
                        json.dump(results, f, indent=2)

elapsed = time.time() - experiment_start
print(f"\nDone! {len(results)} runs in {elapsed / 60:.1f} minutes")
print(f"Results saved to {output_path}")

## Analysis

Now check if pretrain probes correlate with fine-tuning outcomes.

In [ ]:
# Load results
with open(output_path) as f:
    results = json.load(f)

df = pd.DataFrame(results)
print(f"Loaded {len(df)} runs")
print(f"Checkpoints: {sorted(df['checkpoint_step'].unique())}")
print(f"Seeds: {sorted(df['seed'].unique())}")
print(f"\nAccuracy stats:")
print(df['val_accuracy'].describe())

In [ ]:
import warnings

PROBES = [
    'pretrain_gradient_norm',
    'pretrain_gradient_variance',
    'pretrain_sam_sharpness',
    'pretrain_hutchinson_trace',
    'pretrain_top_eigenvalue',
    'pretrain_loss',
]

SHORT = {
    'pretrain_gradient_norm': 'grad_norm',
    'pretrain_gradient_variance': 'grad_var',
    'pretrain_sam_sharpness': 'sam_sharp',
    'pretrain_hutchinson_trace': 'hutch_trace',
    'pretrain_top_eigenvalue': 'top_eig',
    'pretrain_loss': 'mlm_loss',
}


def spearman(x, y):
    valid = ~(np.isnan(x) | np.isnan(y))
    if valid.sum() < 3:
        return np.nan, np.nan, ""
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        r, p = stats.spearmanr(x[valid], y[valid])
    sig = "**" if p < 0.01 else "*" if p < 0.05 else ""
    return r, p, sig

### 1. Pretrain probes vs all fine-tuning accuracies

In [ ]:
print(f"Correlations across all {len(df)} runs:\n")
print(f"{'Probe':<20} {'r':>8} {'p':>10} {'sig':>4}")
print("-" * 45)

for probe in PROBES:
    r, p, sig = spearman(df[probe].values, df['val_accuracy'].values)
    print(f"{SHORT[probe]:<20} {r:>8.3f} {p:>10.4f} {sig:>4}")

### 2. Pretrain probes vs best achievable accuracy (per checkpoint)

In [ ]:
# For each (seed, checkpoint), find the best accuracy across HP configs
best_acc = df.groupby(['seed', 'checkpoint_step']).agg({
    'val_accuracy': 'max',
    **{p: 'first' for p in PROBES},
}).reset_index()

print(f"Pretrain probes vs best accuracy (N = {len(best_acc)}):\n")
print(f"{'Probe':<20} {'r':>8} {'p':>10} {'sig':>4}")
print("-" * 45)

for probe in PROBES:
    r, p, sig = spearman(best_acc[probe].values, best_acc['val_accuracy'].values)
    print(f"{SHORT[probe]:<20} {r:>8.3f} {p:>10.4f} {sig:>4}")

### 3. Pretrain probes vs optimal learning rate

In [ ]:
# For each (seed, checkpoint), find the LR that gave best accuracy
optimal_lr_df = df.loc[df.groupby(['seed', 'checkpoint_step'])['val_accuracy'].idxmax()]

print(f"Pretrain probes vs optimal LR (N = {len(optimal_lr_df)}):\n")
print(f"{'Probe':<20} {'r':>8} {'p':>10} {'sig':>4}")
print("-" * 45)

for probe in PROBES:
    r, p, sig = spearman(optimal_lr_df[probe].values, optimal_lr_df['lr'].values)
    print(f"{SHORT[probe]:<20} {r:>8.3f} {p:>10.4f} {sig:>4}")

print(f"\nOptimal LR per checkpoint:")
for _, row in optimal_lr_df.iterrows():
    print(f"  seed={row['seed']}, {row['checkpoint_step']}: "
          f"lr={row['lr']:.0e}, acc={row['val_accuracy']:.4f}")

### 4. Visualizations

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, probe in enumerate(PROBES):
    ax = axes[i]
    ax.scatter(
        df[probe], df['val_accuracy'],
        c=np.log10(df['lr']), cmap='viridis', alpha=0.6, s=40, edgecolors='gray',
    )
    r, p, sig = spearman(df[probe].values, df['val_accuracy'].values)
    ax.set_title(f"{SHORT[probe]}\nr={r:.3f}{sig} (p={p:.3f})", fontsize=11)
    ax.set_xlabel(SHORT[probe])
    ax.set_ylabel('Val Accuracy')

plt.suptitle('Pretrain Probes vs Fine-tuning Accuracy (color = log10 LR)', fontsize=13, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap
outcomes = ['val_accuracy', 'lr']
probe_labels = [SHORT[p] for p in PROBES]

corr_matrix = np.zeros((len(PROBES), len(outcomes)))
annot = np.empty_like(corr_matrix, dtype=object)

for i, probe in enumerate(PROBES):
    for j, outcome in enumerate(outcomes):
        r, p, sig = spearman(df[probe].values, df[outcome].values)
        corr_matrix[i, j] = r if not np.isnan(r) else 0
        annot[i, j] = f"{r:.2f}{sig}" if not np.isnan(r) else "--"

fig, ax = plt.subplots(figsize=(6, 5))
sns.heatmap(
    corr_matrix, annot=annot, fmt='', cmap='RdBu_r', center=0,
    xticklabels=outcomes, yticklabels=probe_labels, ax=ax,
    vmin=-1, vmax=1,
)
ax.set_title('Pretrain Probe Correlations\n(* p<0.05, ** p<0.01)')
plt.tight_layout()
plt.show()

In [ ]:
# Accuracy distribution by LR
fig, ax = plt.subplots(figsize=(8, 5))
for lr in sorted(df['lr'].unique()):
    subset = df[df['lr'] == lr]
    ax.hist(subset['val_accuracy'], alpha=0.5, label=f'lr={lr:.0e}', bins=15)

ax.set_xlabel('Validation Accuracy')
ax.set_ylabel('Count')
ax.set_title('Accuracy Distribution by Learning Rate')
ax.legend()
plt.tight_layout()
plt.show()

### 5. Summary

In [ ]:
print("=" * 50)
print("SUMMARY")
print("=" * 50)

all_corrs = []
for probe in PROBES:
    r_acc, p_acc, _ = spearman(df[probe].values, df['val_accuracy'].values)
    all_corrs.append(('val_accuracy', probe, r_acc, p_acc))
    r_lr, p_lr, _ = spearman(df[probe].values, df['lr'].values)
    all_corrs.append(('lr', probe, r_lr, p_lr))

all_corrs.sort(key=lambda x: abs(x[2]) if not np.isnan(x[2]) else 0, reverse=True)

print("\nTop correlations:")
for outcome, probe, r, p in all_corrs[:6]:
    sig = "**" if p < 0.01 else "*" if p < 0.05 else ""
    print(f"  {SHORT[probe]} <-> {outcome}: r={r:.3f} (p={p:.4f}) {sig}")

n_sig = sum(1 for _, _, r, p in all_corrs if p < 0.05 and not np.isnan(p))
max_r = max((abs(r) for _, _, r, _ in all_corrs if not np.isnan(r)), default=0)

print(f"\nMax |r|: {max_r:.3f}")
print(f"Significant correlations: {n_sig}/{len(all_corrs)}")

if max_r > 0.5 and n_sig >= 2:
    print("\nVerdict: STRONG signal -- pretrain probes predict fine-tuning outcomes")
elif max_r > 0.3 or n_sig >= 1:
    print("\nVerdict: MODERATE signal -- worth scaling up to full experiment")
else:
    print("\nVerdict: WEAK/NO signal in quick test -- may need more checkpoints for variation")

## Next steps

If the quick test shows signal, scale up by changing the config cell:

```python
CHECKPOINT_STEPS = ["step_200k", "step_600k", "step_1000k", "step_2000k"]
SEEDS = [0, 1, 2, 3, 4]
```

This gives 20 checkpoints x 16 HP configs = 320 runs (~8-10 hours on T4).

The experiment will automatically resume from where it left off thanks to the incremental save.